In [1]:
import numpy as np
import pandas as pd
import data_utils as utils
import re

In [2]:
! ls -la

total 367680
drwxr-xr-x  13 nicolasvelezbeltran  staff       416 Nov 15 20:12 .
drwxr-xr-x  12 nicolasvelezbeltran  staff       384 Nov 15 18:37 ..
drwxr-xr-x   3 nicolasvelezbeltran  staff        96 Nov 15 18:45 .ipynb_checkpoints
drwxr-xr-x   3 nicolasvelezbeltran  staff        96 Nov 15 20:12 __pycache__
-rw-r--r--   1 nicolasvelezbeltran  staff       968 Nov 15 18:37 add_node.py
-rw-r--r--   1 nicolasvelezbeltran  staff      1117 Nov 15 20:12 data_utils.py
-rw-r--r--   1 nicolasvelezbeltran  staff     36368 Nov 15 18:37 exploratory_maps.ipynb
-rw-r--r--   1 nicolasvelezbeltran  staff     16419 Nov 15 20:11 find_shortest_distance.ipynb
-rw-r--r--   1 nicolasvelezbeltran  staff      2229 Nov 15 18:37 generate_data.py
-rw-r--r--   1 nicolasvelezbeltran  staff      1715 Nov 15 18:37 join_aadt.py
-rw-r--r--@  1 nicolasvelezbeltran  staff  28271720 Nov 15 18:37 joined_aadt.csv
-rw-r--r--   1 nicolasvelezbeltran  staff  79604081 Nov 15 18:37 reduced_data.csv
-rw-r--r--@  1 nicolasvelezbel

In [3]:
data = pd.read_csv("joined_aadt.csv")
nodesdf = pd.read_csv("reduced_node_data.csv")

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [31]:
display(data.head())
display(nodesdf.head())

,Road Name,Beginning Description,Ending Description,Count_2014,Count_2015,Count_2016,Count_2017,Count_2018,Station ID,County,...,TruckPercentSourceFieldName,DOT_RegionName,DOT_RegionNumber,DOT_CountyNumber,MorningHighestValue,AfternoonHighestValue,EveningHighestValue,RCSTAYearDirection,SHAPE_Length,geometry
0,BOSTON RD,RT 22 PROVOST AVE,Bronx/West Co Line,23301.0,22471.0,NaN,NaN,21014.0,10009,Bronx,...,NaN,NaN,NaN,NaN,534.0,653.0,719.0,"010009, 2019, NB",811.603299,MULTILINESTRING ((-73.82738679036918 40.886117...
1,BOSTON RD,RT 22 PROVOST AVE,Bronx/West Co Line,23301.0,22471.0,NaN,NaN,21014.0,10009,Bronx,...,NaN,NaN,NaN,NaN,1107.0,1455.0,1557.0,"010009, 2019, CT",811.603299,MULTILINESTRING ((-73.82738679036918 40.886117...
2,BOSTON RD,RT 22 PROVOST AVE,Bronx/West Co Line,23301.0,22471.0,NaN,NaN,21014.0,10009,Bronx,...,NaN,NaN,NaN,NaN,618.0,802.0,852.0,"010009, 2019, SB",811.603299,MULTILINESTRING ((-73.82738679036918 40.886117...
3,BOSTON RD,RT 22 PROVOST AVE,Bronx/West Co Line,23301.0,22471.0,NaN,NaN,21014.0,10009,Bronx,...,NaN,NaN,NaN,NaN,534.0,653.0,719.0,"010009, 2019, NB",863.285819,MULTILINESTRING ((-73.82794249915982 40.885920...
4,BOSTON RD,RT 22 PROVOST AVE,Bronx/West Co Line,23301.0,22471.0,NaN,NaN,21014.0,10009,Bronx,...,NaN,NaN,NaN,NaN,1107.0,1455.0,1557.0,"010009, 2019, CT",863.285819,MULTILINESTRING ((-73.82794249915982 40.885920...


,datetime,longitude,latitude,node
0,2012-07-01 00:05:00,-73.813916,40.697753,42829106
1,2012-07-01 00:05:00,-73.997387,40.762127,42430680
2,2012-07-01 00:10:00,-73.923840,40.733610,597311311
3,2012-07-01 00:10:00,-73.972745,40.588868,42519347
4,2012-07-01 00:20:00,-73.983048,40.677406,42486357


In [35]:
p = re.compile(r'[-+]?[0-9]*\.?[0-9]+')
xy = data['geometry'].apply(lambda x: p.findall(x))

In [15]:
def transform(x): 
    points = []
    j = 0
    point = []
    for i in range(len(x)):
        if j == 0:
            point = [float(x[i])]
            j += 1
        elif j == 1: 
            point.append(float(x[i]))
            points.append(point)
            j = 0
    return np.array(points)
    

In [127]:
def find_min_segment(point,segments): 
    """
    returns index of closest segment. 
    Assumes point is np.array([x,y])
    """
    point = np.array(point)
    def return_min(segment):
        return np.nanmin(utils.lineseg_dists(point,segment[:-1],segment[1:]))
    return np.nanargmin(np.array([return_min(segment) for segment in segments]))

In [124]:
coords = xy.apply(transform)
coords = coords.to_numpy()

In [159]:
df = nodesdf[['longitude','latitude']]

In [164]:
partial = df.groupby(['latitude','longitude']).size().reset_index(name='Freq')


In [169]:
partial.sort_values('Freq',ascending =False)[:20]

,latitude,longitude,Freq
0,0.000000,0.000000,1272
22184,40.608757,-74.038086,670
210296,40.861862,-73.912820,621
97114,40.696033,-73.984530,611
97117,40.696035,-73.984529,585
182214,40.804700,-73.912430,562
163346,40.760601,-73.964314,472
180360,40.798256,-73.827440,462
159701,40.757232,-73.989792,455
19146,40.604153,-74.051980,440
